In [10]:
import pandas as pd
import time
import timeit
import json
import re
import os
import numpy as np
import ast
from pprint import pprint
from pandas.io.json import json_normalize
import pickle

pd.options.display.max_columns = 50

## Kaggle Google Analytics Google Store
### test_v2.csv : visites et transactions de mai à octobre 2018

## Lecture du fichier 
### calcul du temps de lecture 

In [11]:
start_time = time.time()
# 618.3975002765656 seconds

path= r'/home/sophie/Documents/Certif/Input/test_v2.csv'
data_test = pd.read_csv(path,
                   dtype={'fullVisitorId': 'str', 'visitId': 'str'},
                   parse_dates = ['date'])

print('%s seconds' %(time.time() - start_time))

70.35615825653076 seconds


## Exploration des données

In [12]:
data_test.shape

(401589, 13)

In [13]:
data_test.isna().sum()

channelGrouping         0
customDimensions        0
date                    0
device                  0
fullVisitorId           0
geoNetwork              0
hits                    0
socialEngagementType    0
totals                  0
trafficSource           0
visitId                 0
visitNumber             0
visitStartTime          0
dtype: int64

In [14]:
data = data_test.sample(frac = 0.25, random_state=1)

In [15]:
data = data.reset_index(drop=True)

In [16]:
data.shape

(100397, 13)

In [17]:
data.head()  # permet de voir des json imbriqués dans les colonnes device, geoNetwork,hits et totals

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Referral,"[{'index': '4', 'value': 'North America'}]",2018-05-02,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3274048762107183438,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '11',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""3"", ""pageviews"": ""3"",...","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1525286298,1,1525286298
1,Referral,"[{'index': '4', 'value': 'North America'}]",2018-07-22,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3102818095144736797,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""52"", ""pageviews"": ""41...","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1532279455,1,1532279455
2,Organic Search,"[{'index': '4', 'value': 'APAC'}]",2018-05-10,"{""browser"": ""Internet Explorer"", ""browserVersi...",2003235287352658956,"{""continent"": ""Asia"", ""subContinent"": ""Southea...","[{'hitNumber': '1', 'time': '0', 'hour': '2', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1525944774,1,1525944774
3,Affiliates,"[{'index': '4', 'value': 'APAC'}]",2018-07-24,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7907909999860357749,"{""continent"": ""Asia"", ""subContinent"": ""Eastern...","[{'hitNumber': '1', 'time': '0', 'hour': '22',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""referralPath"": ""(not set)"", ""campaign"": ""Dat...",1532496099,1,1532496099
4,Social,"[{'index': '4', 'value': 'APAC'}]",2018-05-13,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",4549822374099137429,"{""continent"": ""Asia"", ""subContinent"": ""Southea...","[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""5"",...","{""referralPath"": ""/yt/music/index.html"", ""camp...",1526216409,1,1526216409


In [21]:
data.dtypes

channelGrouping                 object
customDimensions                object
date                    datetime64[ns]
device                          object
fullVisitorId                   object
geoNetwork                      object
hits                            object
socialEngagementType            object
totals                          object
trafficSource                   object
visitId                         object
visitNumber                      int64
visitStartTime                   int64
dtype: object

## device: beaucoup d'attributs peu renseignés 
### à garder: "browser", "deviceCategory",  "OperatingSystem"


### transforme le str en dictionnaire 

In [18]:
df_device = data['device']
df_device.value_counts()

{"browser": "Chrome", "browserVersion": "not available in demo dataset", "browserSize": "not available in demo dataset", "operatingSystem": "Windows", "operatingSystemVersion": "not available in demo dataset", "isMobile": false, "mobileDeviceBranding": "not available in demo dataset", "mobileDeviceModel": "not available in demo dataset", "mobileInputSelector": "not available in demo dataset", "mobileDeviceInfo": "not available in demo dataset", "mobileDeviceMarketingName": "not available in demo dataset", "flashVersion": "not available in demo dataset", "language": "not available in demo dataset", "screenColors": "not available in demo dataset", "screenResolution": "not available in demo dataset", "deviceCategory": "desktop"}                                                    29485
{"browser": "Chrome", "browserVersion": "not available in demo dataset", "browserSize": "not available in demo dataset", "operatingSystem": "Macintosh", "operatingSystemVersion": "not available in demo datas

In [20]:
jdevice = df_device.apply(json.loads) # json loads -> returns an object from a string representing a json object
jdevicelist = pd.DataFrame(jdevice.tolist()) # build a df from a list
jdevicelist

,browser,browserVersion,browserSize,operatingSystem,operatingSystemVersion,isMobile,mobileDeviceBranding,mobileDeviceModel,mobileInputSelector,mobileDeviceInfo,mobileDeviceMarketingName,flashVersion,language,screenColors,screenResolution,deviceCategory
0,Chrome,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
1,Chrome,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
2,Internet Explorer,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,tablet
3,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
4,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100392,Safari,not available in demo dataset,not available in demo dataset,iOS,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile
100393,Chrome,not available in demo dataset,not available in demo dataset,Linux,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
100394,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
100395,Chrome,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop


In [21]:
jdevicelistf = jdevicelist[['browser', 'deviceCategory', 'operatingSystem']]

In [22]:
jdevicelistf.shape

(100397, 3)

In [23]:
jdevicelistf.head()

,browser,deviceCategory,operatingSystem
0,Chrome,desktop,Macintosh
1,Chrome,desktop,Macintosh
2,Internet Explorer,tablet,Windows
3,Chrome,desktop,Windows
4,Chrome,desktop,Windows


In [24]:
data = pd.concat([data,jdevicelistf], axis=1)

In [25]:
data.shape

(100397, 16)

In [26]:
data.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime,browser,deviceCategory,operatingSystem
0,Referral,"[{'index': '4', 'value': 'North America'}]",2018-05-02,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3274048762107183438,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '11',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""3"", ""pageviews"": ""3"",...","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1525286298,1,1525286298,Chrome,desktop,Macintosh
1,Referral,"[{'index': '4', 'value': 'North America'}]",2018-07-22,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3102818095144736797,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""52"", ""pageviews"": ""41...","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1532279455,1,1532279455,Chrome,desktop,Macintosh
2,Organic Search,"[{'index': '4', 'value': 'APAC'}]",2018-05-10,"{""browser"": ""Internet Explorer"", ""browserVersi...",2003235287352658956,"{""continent"": ""Asia"", ""subContinent"": ""Southea...","[{'hitNumber': '1', 'time': '0', 'hour': '2', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1525944774,1,1525944774,Internet Explorer,tablet,Windows
3,Affiliates,"[{'index': '4', 'value': 'APAC'}]",2018-07-24,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7907909999860357749,"{""continent"": ""Asia"", ""subContinent"": ""Eastern...","[{'hitNumber': '1', 'time': '0', 'hour': '22',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""referralPath"": ""(not set)"", ""campaign"": ""Dat...",1532496099,1,1532496099,Chrome,desktop,Windows
4,Social,"[{'index': '4', 'value': 'APAC'}]",2018-05-13,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",4549822374099137429,"{""continent"": ""Asia"", ""subContinent"": ""Southea...","[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""5"",...","{""referralPath"": ""/yt/music/index.html"", ""camp...",1526216409,1,1526216409,Chrome,desktop,Windows


In [27]:
data.drop('device', axis=1, inplace=True)
data.head()

,channelGrouping,customDimensions,date,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime,browser,deviceCategory,operatingSystem
0,Referral,"[{'index': '4', 'value': 'North America'}]",2018-05-02,3274048762107183438,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '11',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""3"", ""pageviews"": ""3"",...","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1525286298,1,1525286298,Chrome,desktop,Macintosh
1,Referral,"[{'index': '4', 'value': 'North America'}]",2018-07-22,3102818095144736797,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""52"", ""pageviews"": ""41...","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1532279455,1,1532279455,Chrome,desktop,Macintosh
2,Organic Search,"[{'index': '4', 'value': 'APAC'}]",2018-05-10,2003235287352658956,"{""continent"": ""Asia"", ""subContinent"": ""Southea...","[{'hitNumber': '1', 'time': '0', 'hour': '2', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1525944774,1,1525944774,Internet Explorer,tablet,Windows
3,Affiliates,"[{'index': '4', 'value': 'APAC'}]",2018-07-24,7907909999860357749,"{""continent"": ""Asia"", ""subContinent"": ""Eastern...","[{'hitNumber': '1', 'time': '0', 'hour': '22',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""referralPath"": ""(not set)"", ""campaign"": ""Dat...",1532496099,1,1532496099,Chrome,desktop,Windows
4,Social,"[{'index': '4', 'value': 'APAC'}]",2018-05-13,4549822374099137429,"{""continent"": ""Asia"", ""subContinent"": ""Southea...","[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""5"",...","{""referralPath"": ""/yt/music/index.html"", ""camp...",1526216409,1,1526216409,Chrome,desktop,Windows


## Transformer geoNetwork
### garder continent subContinent country region city

In [28]:
jgeo = data['geoNetwork'].apply(json.loads)
jgeolist = pd.DataFrame(jgeo.tolist())
jgeolist

,continent,subContinent,country,region,metro,city,cityId,networkDomain,latitude,longitude,networkLocation
0,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Mountain View,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
1,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Sunnyvale,not available in demo dataset,comcast.net,not available in demo dataset,not available in demo dataset,not available in demo dataset
2,Asia,Southeast Asia,Singapore,(not set),(not set),Singapore,not available in demo dataset,pembertoninvestments.com,not available in demo dataset,not available in demo dataset,not available in demo dataset
3,Asia,Eastern Asia,Taiwan,New Taipei City,(not set),(not set),not available in demo dataset,hinet.net,not available in demo dataset,not available in demo dataset,not available in demo dataset
4,Asia,Southeast Asia,Indonesia,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,telkom.net.id,not available in demo dataset,not available in demo dataset,not available in demo dataset
...,...,...,...,...,...,...,...,...,...,...,...
100392,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
100393,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,rr.com,not available in demo dataset,not available in demo dataset,not available in demo dataset
100394,Asia,Southern Asia,Bangladesh,Dhaka Division,(not set),(not set),not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset
100395,Europe,Southern Europe,Spain,Catalonia,(not set),Barcelona,not available in demo dataset,prismasl.com,not available in demo dataset,not available in demo dataset,not available in demo dataset


In [29]:
%who_ls DataFrame

['data', 'data_test', 'jdevicelist', 'jdevicelistf', 'jgeolist']

In [30]:
data = pd.concat([data,jgeolist['continent'], jgeolist['subContinent'], jgeolist['country'], jgeolist['region'], jgeolist['city']], axis=1) 

In [31]:
data.drop('geoNetwork', axis=1, inplace=True)
data.head()

,channelGrouping,customDimensions,date,fullVisitorId,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime,browser,deviceCategory,operatingSystem,continent,subContinent,country,region,city
0,Referral,"[{'index': '4', 'value': 'North America'}]",2018-05-02,3274048762107183438,"[{'hitNumber': '1', 'time': '0', 'hour': '11',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""3"", ""pageviews"": ""3"",...","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1525286298,1,1525286298,Chrome,desktop,Macintosh,Americas,Northern America,United States,California,Mountain View
1,Referral,"[{'index': '4', 'value': 'North America'}]",2018-07-22,3102818095144736797,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""52"", ""pageviews"": ""41...","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1532279455,1,1532279455,Chrome,desktop,Macintosh,Americas,Northern America,United States,California,Sunnyvale
2,Organic Search,"[{'index': '4', 'value': 'APAC'}]",2018-05-10,2003235287352658956,"[{'hitNumber': '1', 'time': '0', 'hour': '2', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1525944774,1,1525944774,Internet Explorer,tablet,Windows,Asia,Southeast Asia,Singapore,(not set),Singapore
3,Affiliates,"[{'index': '4', 'value': 'APAC'}]",2018-07-24,7907909999860357749,"[{'hitNumber': '1', 'time': '0', 'hour': '22',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""referralPath"": ""(not set)"", ""campaign"": ""Dat...",1532496099,1,1532496099,Chrome,desktop,Windows,Asia,Eastern Asia,Taiwan,New Taipei City,(not set)
4,Social,"[{'index': '4', 'value': 'APAC'}]",2018-05-13,4549822374099137429,"[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""5"",...","{""referralPath"": ""/yt/music/index.html"", ""camp...",1526216409,1,1526216409,Chrome,desktop,Windows,Asia,Southeast Asia,Indonesia,not available in demo dataset,not available in demo dataset


In [32]:
data.shape

(100397, 19)

## Transformer totals
### et garder tous les champs

In [33]:
jtot = data['totals'].apply(json.loads)
jtotlist = pd.DataFrame(jtot.tolist())

jtotlist.head()

,visits,hits,pageviews,timeOnSite,newVisits,sessionQualityDim,transactions,transactionRevenue,totalTransactionRevenue,bounces
0,1,3,3,23,1,1,NaN,NaN,NaN,NaN
1,1,52,41,995,1,66,1,62340000,67340000,NaN
2,1,1,1,NaN,1,1,NaN,NaN,NaN,1
3,1,1,1,NaN,1,1,NaN,NaN,NaN,1
4,1,8,5,80,1,1,NaN,NaN,NaN,NaN


In [34]:
jtotlist.isna().sum()

visits                         0
hits                           0
pageviews                     39
timeOnSite                 45890
newVisits                  28870
sessionQualityDim              0
transactions               98861
transactionRevenue         99272
totalTransactionRevenue    99272
bounces                    54740
dtype: int64

In [35]:
%who_ls DataFrame

['data', 'data_test', 'jdevicelist', 'jdevicelistf', 'jgeolist', 'jtotlist']

In [36]:
jtotlist['bounces'] = pd.to_numeric(jtotlist['bounces'], errors='coerce')
jtotlist['newVisits'] = pd.to_numeric(jtotlist['newVisits'], errors='coerce')
jtotlist['timeOnSite'] = pd.to_numeric(jtotlist['timeOnSite'], errors='coerce')
jtotlist['sessionQualityDim'] = pd.to_numeric(jtotlist['sessionQualityDim'], errors='coerce')
jtotlist['totalTransactionRevenue'] = pd.to_numeric(jtotlist['totalTransactionRevenue'], errors='coerce')
jtotlist['transactions'] = pd.to_numeric(jtotlist['transactions'], errors='coerce')
jtotlist['pageviews'] = pd.to_numeric(jtotlist['pageviews'], errors='coerce')

In [37]:
jtotlist.dtypes

visits                      object
hits                        object
pageviews                  float64
timeOnSite                 float64
newVisits                  float64
sessionQualityDim            int64
transactions               float64
transactionRevenue          object
totalTransactionRevenue    float64
bounces                    float64
dtype: object

In [38]:
data = pd.concat([data, jtotlist['visits'].astype('int'), jtotlist['newVisits'], jtotlist['bounces'], jtotlist['hits'].astype('int'), jtotlist['pageviews'], jtotlist['sessionQualityDim'], jtotlist['timeOnSite'], jtotlist['totalTransactionRevenue'], jtotlist['transactions']], axis=1) 

In [39]:
data.dtypes

channelGrouping                    object
customDimensions                   object
date                       datetime64[ns]
fullVisitorId                      object
hits                               object
socialEngagementType               object
totals                             object
trafficSource                      object
visitId                            object
visitNumber                         int64
visitStartTime                      int64
browser                            object
deviceCategory                     object
operatingSystem                    object
continent                          object
subContinent                       object
country                            object
region                             object
city                               object
visits                              int64
newVisits                         float64
bounces                           float64
hits                                int64
pageviews                         

In [40]:
data.drop('totals', axis=1, inplace=True)


In [41]:
data.drop(['customDimensions', 'trafficSource'], axis=1, inplace=True)

In [42]:
data.head()

,channelGrouping,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,browser,deviceCategory,operatingSystem,continent,subContinent,country,region,city,visits,newVisits,bounces,hits,pageviews,sessionQualityDim,timeOnSite,totalTransactionRevenue,transactions
0,Referral,2018-05-02,3274048762107183438,"[{'hitNumber': '1', 'time': '0', 'hour': '11',...",Not Socially Engaged,1525286298,1,1525286298,Chrome,desktop,Macintosh,Americas,Northern America,United States,California,Mountain View,1,1.0,NaN,3,3.0,1,23.0,NaN,NaN
1,Referral,2018-07-22,3102818095144736797,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,1532279455,1,1532279455,Chrome,desktop,Macintosh,Americas,Northern America,United States,California,Sunnyvale,1,1.0,NaN,52,41.0,66,995.0,67340000.0,1.0
2,Organic Search,2018-05-10,2003235287352658956,"[{'hitNumber': '1', 'time': '0', 'hour': '2', ...",Not Socially Engaged,1525944774,1,1525944774,Internet Explorer,tablet,Windows,Asia,Southeast Asia,Singapore,(not set),Singapore,1,1.0,1.0,1,1.0,1,NaN,NaN,NaN
3,Affiliates,2018-07-24,7907909999860357749,"[{'hitNumber': '1', 'time': '0', 'hour': '22',...",Not Socially Engaged,1532496099,1,1532496099,Chrome,desktop,Windows,Asia,Eastern Asia,Taiwan,New Taipei City,(not set),1,1.0,1.0,1,1.0,1,NaN,NaN,NaN
4,Social,2018-05-13,4549822374099137429,"[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",Not Socially Engaged,1526216409,1,1526216409,Chrome,desktop,Windows,Asia,Southeast Asia,Indonesia,not available in demo dataset,not available in demo dataset,1,1.0,NaN,8,5.0,1,80.0,NaN,NaN


In [43]:
column_numbers = [x for x in range(data.shape[1])]  # list of columns' integer indices
print(column_numbers)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [44]:
column_numbers.remove(3) #removing column integer index 3
data1 = data.iloc[:, column_numbers] #return all columns except the 3rd column

In [45]:
data1.head()

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,browser,deviceCategory,operatingSystem,continent,subContinent,country,region,city,visits,newVisits,bounces,hits,pageviews,sessionQualityDim,timeOnSite,totalTransactionRevenue,transactions
0,Referral,2018-05-02,3274048762107183438,Not Socially Engaged,1525286298,1,1525286298,Chrome,desktop,Macintosh,Americas,Northern America,United States,California,Mountain View,1,1.0,NaN,3,3.0,1,23.0,NaN,NaN
1,Referral,2018-07-22,3102818095144736797,Not Socially Engaged,1532279455,1,1532279455,Chrome,desktop,Macintosh,Americas,Northern America,United States,California,Sunnyvale,1,1.0,NaN,52,41.0,66,995.0,67340000.0,1.0
2,Organic Search,2018-05-10,2003235287352658956,Not Socially Engaged,1525944774,1,1525944774,Internet Explorer,tablet,Windows,Asia,Southeast Asia,Singapore,(not set),Singapore,1,1.0,1.0,1,1.0,1,NaN,NaN,NaN
3,Affiliates,2018-07-24,7907909999860357749,Not Socially Engaged,1532496099,1,1532496099,Chrome,desktop,Windows,Asia,Eastern Asia,Taiwan,New Taipei City,(not set),1,1.0,1.0,1,1.0,1,NaN,NaN,NaN
4,Social,2018-05-13,4549822374099137429,Not Socially Engaged,1526216409,1,1526216409,Chrome,desktop,Windows,Asia,Southeast Asia,Indonesia,not available in demo dataset,not available in demo dataset,1,1.0,NaN,8,5.0,1,80.0,NaN,NaN


In [46]:
print(data1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100397 entries, 0 to 100396
Data columns (total 24 columns):
channelGrouping            100397 non-null object
date                       100397 non-null datetime64[ns]
fullVisitorId              100397 non-null object
socialEngagementType       100397 non-null object
visitId                    100397 non-null object
visitNumber                100397 non-null int64
visitStartTime             100397 non-null int64
browser                    100397 non-null object
deviceCategory             100397 non-null object
operatingSystem            100397 non-null object
continent                  100397 non-null object
subContinent               100397 non-null object
country                    100397 non-null object
region                     100397 non-null object
city                       100397 non-null object
visits                     100397 non-null int64
newVisits                  71527 non-null float64
bounces                    45657 n

## Sauvegarde des données hors hits

In [47]:
def session_save(picke_path, objects):
    pickling_on = open(picke_path,"wb")
    try:
        pickle.dump(objects, pickling_on)
        print("session_save success\n")
    except:
        print("session_save failed\n")
    finally:
        pickling_on.close()

In [48]:
session_save('SavePickle/data1_test25p.pickle', data1)

session_save success



## Traiter Hits - focus produits pour avoir les produits consultés par session et constituer une table produits

In [49]:
data2 = data[['date', 'fullVisitorId', 'hits', 'visitId']]

In [50]:
data2.head()

,date,fullVisitorId,hits,hits,visitId
0,2018-05-02,3274048762107183438,"[{'hitNumber': '1', 'time': '0', 'hour': '11',...",3,1525286298
1,2018-07-22,3102818095144736797,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",52,1532279455
2,2018-05-10,2003235287352658956,"[{'hitNumber': '1', 'time': '0', 'hour': '2', ...",1,1525944774
3,2018-07-24,7907909999860357749,"[{'hitNumber': '1', 'time': '0', 'hour': '22',...",1,1532496099
4,2018-05-13,4549822374099137429,"[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",8,1526216409


In [51]:
column_numbers = [x for x in range(data2.shape[1])]  # list of columns' integer indices
print(column_numbers)

[0, 1, 2, 3, 4]


In [52]:
column_numbers.remove(3) #removing column integer index 3 - using index as there are 2 columns "hits"
data2 = data2.iloc[:, column_numbers] #return all columns except the 3th column

In [53]:
data2.shape

(100397, 4)

In [54]:
data2.head()

,date,fullVisitorId,hits,visitId
0,2018-05-02,3274048762107183438,"[{'hitNumber': '1', 'time': '0', 'hour': '11',...",1525286298
1,2018-07-22,3102818095144736797,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",1532279455
2,2018-05-10,2003235287352658956,"[{'hitNumber': '1', 'time': '0', 'hour': '2', ...",1525944774
3,2018-07-24,7907909999860357749,"[{'hitNumber': '1', 'time': '0', 'hour': '22',...",1532496099
4,2018-05-13,4549822374099137429,"[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",1526216409


In [55]:
hits = data2['hits'].head()
hitsj = hits.to_dict()
hitsj1 = hitsj[1]
hitNumber_list = ast.literal_eval(hitsj1)
hitNumber_list

[{'hitNumber': '1',
  'time': '0',
  'hour': '10',
  'minute': '10',
  'isInteraction': True,
  'isEntrance': True,
  'referer': 'https://mall.googleplex.com/',
  'page': {'pagePath': '/home',
   'hostname': 'shop.googlemerchandisestore.com',
   'pageTitle': 'Home',
   'pagePathLevel1': '/home',
   'pagePathLevel2': '',
   'pagePathLevel3': '',
   'pagePathLevel4': ''},
  'appInfo': {'screenName': 'shop.googlemerchandisestore.com/home',
   'landingScreenName': 'shop.googlemerchandisestore.com/home',
   'exitScreenName': 'shop.googlemerchandisestore.com/ordercompleted.html',
   'screenDepth': '0'},
  'exceptionInfo': {'isFatal': True},
  'product': [],
  'promotion': [{'promoId': 'Colorful Google Tees',
    'promoName': 'Colorful Google Tees',
    'promoCreative': "Category - Men's Apparel",
    'promoPosition': 'Row 1'},
   {'promoId': 'New Snapback Hats',
    'promoName': 'New Snapback Hats',
    'promoCreative': 'Category - Headgear',
    'promoPosition': 'Row 2'},
   {'promoId': 'Go

In [56]:
hitNumber_list[1]['product']

[{'productSKU': 'GGOEGXXX1041',
  'v2ProductName': 'Google Tonal Tee Fog',
  'v2ProductCategory': "Home/Apparel/Men's/",
  'productVariant': '(not set)',
  'productBrand': '(not set)',
  'productPrice': '21990000',
  'localProductPrice': '21990000',
  'isImpression': True,
  'customDimensions': [],
  'customMetrics': [],
  'productListName': 'Category',
  'productListPosition': '1',
  'productCouponCode': '(not set)'},
 {'productSKU': 'GGOEGXXX1042',
  'v2ProductName': 'Google Tonal Tee Spearmint',
  'v2ProductCategory': "Home/Apparel/Men's/",
  'productVariant': '(not set)',
  'productBrand': '(not set)',
  'productPrice': '21990000',
  'localProductPrice': '21990000',
  'isImpression': True,
  'customDimensions': [],
  'customMetrics': [],
  'productListName': 'Category',
  'productListPosition': '2',
  'productCouponCode': '(not set)'},
 {'productSKU': 'GGOEGXXX1043',
  'v2ProductName': 'Google Tonal Tee Coral',
  'v2ProductCategory': "Home/Apparel/Men's/",
  'productVariant': '(not

In [57]:
data2['hits'] = data2.hits.apply(lambda x: ast.literal_eval(x)) # to convert a string into a dictionary

In [58]:
all_skus_in_order = [] #liste des sku par visite, quel que soit le nombre de hits
all_products = [] #liste des produits avec leurs attributs

for index, row in data2.iterrows(): #pour chaque ligne
    temp = []
    hits = row['hits']
    for hit in hits: #pour chaque hit
        if len(hit['product']) > 0: #si le champ product n'est pas vide
            for product in hit['product']: #pour chaque produit
                temp.append(product['productSKU']) #garde le sku dans liste temporaire
                all_products.append(product) #ajoute le produit dans all_products
    all_skus_in_order.append(temp) #ajoute la liste de skus pour cetteligne a la liste principale

data2['tous_skus'] = all_skus_in_order #la liste devient colonne car l'ordre est le même
all_products = list({item["productSKU"]: item for item in all_products}.values()) #assurer unicité des produits sur le sku
produits = pd.DataFrame(all_products) #dataframe produits

In [59]:
data2.head()

,date,fullVisitorId,hits,visitId,tous_skus
0,2018-05-02,3274048762107183438,"[{'hitNumber': '1', 'time': '0', 'hour': '11',...",1525286298,[]
1,2018-07-22,3102818095144736797,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",1532279455,"[GGOEGXXX1041, GGOEGXXX1042, GGOEGXXX1043, GGO..."
2,2018-05-10,2003235287352658956,"[{'hitNumber': '1', 'time': '0', 'hour': '2', ...",1525944774,[]
3,2018-07-24,7907909999860357749,"[{'hitNumber': '1', 'time': '0', 'hour': '22',...",1532496099,[]
4,2018-05-13,4549822374099137429,"[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",1526216409,"[GGOEYDHB098499, GGOEYHBB102199, GGOEYOCB09269..."


In [60]:
data2.drop('hits', axis = 1, inplace = True)

In [61]:
product_nav = data2.explode('tous_skus') # constitution d'un df avec un produit pour une session par ligne

In [62]:
product_nav.shape

(2102530, 4)

In [63]:
product_nav.head()

,date,fullVisitorId,visitId,tous_skus
0,2018-05-02,3274048762107183438,1525286298,NaN
1,2018-07-22,3102818095144736797,1532279455,GGOEGXXX1041
1,2018-07-22,3102818095144736797,1532279455,GGOEGXXX1042
1,2018-07-22,3102818095144736797,1532279455,GGOEGXXX1043
1,2018-07-22,3102818095144736797,1532279455,GGOEGXXX1044


In [64]:
product_nav.isna().sum()

date                 0
fullVisitorId        0
visitId              0
tous_skus        36704
dtype: int64

In [65]:
product_nav.tous_skus.value_counts()

GGOEYDHB086099    38381
GGOEYOCB092699    37298
GGOEGABL100799    36597
GGOEGXXX0918      35928
GGOEYHBB102199    35801
                  ...  
GGOEGAXC088928        1
GGOEGAEJ104118        1
GGOEGAAX0295          1
GGOEYFKQ020699        1
GGOEAALJ091414        1
Name: tous_skus, Length: 606, dtype: int64

In [66]:
produits.shape

(606, 15)

In [67]:
produits.head()

,productSKU,v2ProductName,v2ProductCategory,productVariant,productBrand,productPrice,localProductPrice,isImpression,customDimensions,customMetrics,productListName,productListPosition,productCouponCode,productQuantity,isClick
0,GGOEGXXX1041,Google Tonal Tee Fog,Home/Apparel/Men's/Men's T-Shirts/,(not set),(not set),21990000,21990000,True,[],[],Category,1,(not set),NaN,NaN
1,GGOEGXXX1042,Google Tonal Tee Spearmint,Home/Apparel/Men's/Men's T-Shirts/,(not set),(not set),21990000,21990000,True,[],[],Category,2,(not set),NaN,NaN
2,GGOEGXXX1043,Google Tonal Tee Coral,Home/Apparel/Men's/Men's T-Shirts/,(not set),(not set),21990000,21990000,True,[],[],Category,3,(not set),NaN,NaN
3,GGOEGXXX1044,Google Tonal Tee Slate Grey,Home/Apparel/Men's/Men's T-Shirts/,(not set),(not set),21990000,21990000,True,[],[],Category,4,(not set),NaN,NaN
4,GGOEAXXX1038,Android Unisex Pride Tee,Home/Apparel/Men's/Men's T-Shirts/,(not set),(not set),23990000,23990000,True,[],[],Category,5,(not set),NaN,NaN


In [68]:
produits = produits[['productSKU', 'v2ProductName', 'v2ProductCategory', 'productPrice']]

In [69]:
produits.head()

,productSKU,v2ProductName,v2ProductCategory,productPrice
0,GGOEGXXX1041,Google Tonal Tee Fog,Home/Apparel/Men's/Men's T-Shirts/,21990000
1,GGOEGXXX1042,Google Tonal Tee Spearmint,Home/Apparel/Men's/Men's T-Shirts/,21990000
2,GGOEGXXX1043,Google Tonal Tee Coral,Home/Apparel/Men's/Men's T-Shirts/,21990000
3,GGOEGXXX1044,Google Tonal Tee Slate Grey,Home/Apparel/Men's/Men's T-Shirts/,21990000
4,GGOEAXXX1038,Android Unisex Pride Tee,Home/Apparel/Men's/Men's T-Shirts/,23990000


In [70]:
produits.describe()

,productSKU,v2ProductName,v2ProductCategory,productPrice
count,606,606,606,606
unique,606,212,35,61
top,9190301,Android Unisex Pride Tee,(not set),0
freq,1,10,232,151


## Sauvegarde pour constituer les tables product_nav et product

In [71]:
session_save(r"SavePickle/product_nav.pickle", product_nav)

session_save success



In [72]:
session_save(r"SavePickle/produits.pickle", produits)

session_save success

